# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Juan Carlos Lopez Gonzalez <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>


Descripción del problema:(copiar enunciado)


Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.








                                        

#Modelo
**- ¿Como represento el espacio de soluciones?**
Las soluciones están representadas en un diccionario que es imprimido en pantalla cuya clave será el dia y cuyo valor será una lista con las diferentes tomas que se graben ese dia.


**- ¿Cual es la función objetivo?**
La función objetivo está implementada en la función select_nodo(), y se intenta obtener en todo momento la primera toma que menos coste instroduzca en el día, ya que el coste de cada actor por ir al día es de 1, independientemente del numero de tomas que grabe, por lo que hay que maximizar la agrupación de tomas que usen los mismos actores.


**- ¿Como implemento las restricciones?**
La única restricción ha sido no grabar más de 6 tomas por día

In [ ]:
#Respuesta

import pandas as pd
from google.colab import files
import numpy as np


# Subir el archivo CSV
archivo = files.upload()

# Seleccionar el archivo
datos = list(archivo.keys())[0]

# Cargar el dataframe desde el archivo CSV
df = pd.read_csv(datos, index_col=0)
df.drop(df.columns[10], axis=1, inplace=True)
df_cleaned = df.iloc[1:31, 0:11]

nuevas_columnas = {columna: f'Actor {i+1}' for i, columna in enumerate(df_cleaned.columns)}
df_cleaned = df_cleaned.rename(columns=nuevas_columnas)
df_cleaned.rename(columns={'Actor 11': 'Total'}, inplace=True)


print(df_cleaned)



Saving Datos problema doblaje(30 tomas, 10 actores) - Hoja 1.csv to Datos problema doblaje(30 tomas, 10 actores) - Hoja 1 (25).csv
    Actor 1  Actor 2  Actor 3  Actor 4  Actor 5  Actor 6  Actor 7  Actor 8  \
1       1.0      1.0      1.0      1.0      1.0      0.0      0.0      0.0   
2       0.0      0.0      1.0      1.0      1.0      0.0      0.0      0.0   
3       0.0      1.0      0.0      0.0      1.0      0.0      1.0      0.0   
4       1.0      1.0      0.0      0.0      0.0      0.0      1.0      1.0   
5       0.0      1.0      0.0      1.0      0.0      0.0      0.0      1.0   
6       1.0      1.0      0.0      1.0      1.0      0.0      0.0      0.0   
7       1.0      1.0      0.0      1.0      1.0      0.0      0.0      0.0   
8       1.0      1.0      0.0      0.0      0.0      1.0      0.0      0.0   
9       1.0      1.0      0.0      1.0      0.0      0.0      0.0      0.0   
10      1.0      1.0      0.0      0.0      0.0      1.0      0.0      0.0   
11      1.0

In [ ]:



costo_total=0


#Pasa el dataset a un diccionario para facilitar el manejo de los datos
def dataset_a_diccionario(dataset):
    diccionario = {}
    for index, row in dataset.iterrows():
        actores = [int(actor.split()[1]) for actor, valor in row[:-1].items() if valor == 1.0]
        diccionario[index] = actores
    return diccionario

#Calcula el costo del nodo respecto a otro nodo, para ello se compara el numero de actores distintos de las tramas diarias con las posibles nuevas
def calcular_costo_nodo(principal, secundario):
    costo = 0
    for actor in secundario:
        if actor not in principal:
            costo += 1
    return costo


#Selecciona los 5 nodos hijos en función del padre (clave del diccionario) que se le pase

def select_nodo(clave):
  global costo_total

  #Variable para obtener los distintos actores de la escena
  numero_de_actores_escena = diccionario_actores[clave]
  #Tomas grabadas
  tomas_diarias=[]

  #Se inicializa el padre, eliminandolo del diccionario
  tomas_diarias.append(clave)
  del diccionario_actores[clave]
  diccionario_costos={}

  #Este bucle se ejecutara mientras no tengamos 6 tomas y el diccionario de actores no este vacio
  while(len(tomas_diarias)!=6 and len(diccionario_actores) > 0):

    #Obtiene los costos de todas las tomas y las guarda en un diccionario
    for c, v in diccionario_actores.items():
      costo=calcular_costo_nodo(numero_de_actores_escena,v)
      diccionario_costos[c]= costo

    #Obtiene el nodo de menor costo
    clave_menor_costo = min(diccionario_actores, key=diccionario_actores.get)
    actores_menor_costo = diccionario_actores[clave_menor_costo]

    #nuevos actores de la toma en la escena
    for actor in actores_menor_costo:
      if actor not in numero_de_actores_escena:
        numero_de_actores_escena.append(actor)

    #Se añade al numero de tomas
    tomas_diarias.append(clave_menor_costo)
    del diccionario_actores[clave_menor_costo]
    diccionario_costos={}

  #Una vez tenemos las 6 tramas obtenemos el coste diario de los actores (actores distintos ese dia) y lo sumamos al global
  coste_diario=len(numero_de_actores_escena)
  costo_total+=coste_diario

  return tomas_diarias


#Esta función imprimirá el diccionario y el costo total
def imprimir_diccionario(diccionario):
    print("     PLANIFICACIÓN DIARIA DE LAS TOMAS")
    print("--------------------------------------------------")

    for dia, (clave, valores) in enumerate(diccionario.items(), start=1):
        print(f"  Dia {dia}: Tomas {valores}")

    print("--------------------------------------------------")

    print("--------------------------------------------------")
    print("Costo total:", costo_total)
    print("--------------------------------------------------")



#Función que obtiene la planificacion diaria final
def planificacion():
  #diccionario resultado
  dict_planificacion = {}
  dia = 1

  #Obtendra las tramas diarias mientras el diccionario tenga alguna trama
  while len(diccionario_actores) > 0:
    #Usa como padre el primer valor del diccionario
    primer_clave = next(iter(diccionario_actores))
    tomas_diarias = select_nodo(primer_clave)
    dict_planificacion[str(dia)] = tomas_diarias
    dia += 1

  imprimir_diccionario(dict_planificacion)


diccionario_actores = dataset_a_diccionario(df_cleaned)


planificacion()




     PLANIFICACIÓN DIARIA DE LAS TOMAS
--------------------------------------------------
  Dia 1: Tomas ['1', '22', '12', '11', '9', '6']
  Dia 2: Tomas ['2', '7', '25', '8', '10', '15']
  Dia 3: Tomas ['3', '4', '17', '19', '23', '20']
  Dia 4: Tomas ['5', '26', '14', '28', '30', '13']
  Dia 5: Tomas ['16', '29', '18', '24', '27', '21']
--------------------------------------------------
--------------------------------------------------
Costo total: 38
--------------------------------------------------


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta

El orden de complejidad es de O(n^2) y el espacio de soluciones se puede considerar razonable porque a pesar de usar una gran cantidad de memoria para agrupar las posibles combinaciones, solo se consideran un subconjunto de las mismas por lo que no es necesario guardarlas en memoria


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta

He utilizado un algoritmo heurístico de busqueda local para solucionar el problema. En primer lugar la he escogido porque era bastante facil de diseñar, ya que es muy sencillo pensar en escoger la mejor opción a cada momento. Tambien he considerado que podría aportar una buena solución, ya que si o si iba a encontrar una solución aunque no fuera la más óptima y además, al escoger en todo momento la mejor opción, en cierta parte nos aseguramos de que la solución es cercana a la óptima.